# ⓵ Useful Imports

In [4]:
# Web ＠
import requests
import json

# Visualisation 📊
import csv
import pandas as pd

# Time 🕦
import time
import datetime

In [5]:
def Data_From_Posts(number_of_posts,outfile_name):
    
    with open(outfile_name+'_posts.json', 'r') as f:
        arr = json.loads(f.read()) # load json data from previous step
    
    print("Opening of "+outfile_name+"_posts.json successful")  
    posts_data = []
    coms= []
    url_imgs= []
    hashtags= []
    number_of_posts_limit=number_of_posts
    number_of_posts=0
    successful_requests=0
    failed_requests=0
    kicked_requests=0
    print("Start retrieving posts data")
    for item in arr:
        if(number_of_posts<number_of_posts_limit):
            shortcode = item['shortcode']
            url = "https://www.instagram.com/p/{0}/?__a=1".format(shortcode)
            while True:
                try:
                    r = requests.get(url,timeout = 5)
                    time.sleep(500)
                except Exception as e:
                    continue
                break
            if ( r.status_code==200):
                number_of_posts+=1
                successful_requests+=1
                data = json.loads(r.text)
                #print(data)
                try:
                    location = data['graphql']['shortcode_media']['location']['name'] # get location for a post

                except:
                     location = '' # if location is NULL


                likes=data['graphql']['shortcode_media']['edge_media_preview_like']['count'] #ok

                try :
                    number_coms=data['graphql']['shortcode_media']['edge_media_to_parent_comment']['count'] #ok
                except :
                    number_coms=data['graphql']['shortcode_media']['edge_media_to_comment']['count']

                coms=[]

                try :
                    for nodes in data['graphql']['shortcode_media']['edge_media_to_parent_comment']['edges']:
                        coms.append(nodes['node']['text'])
                except:
                    try : 
                        for nodes in data['graphql']['shortcode_media']['edge_media_to_comment']['edges']:
                            coms.append(nodes['node']['text'])

                    except : 
                        coms.append("")

                url_img=data['graphql']['shortcode_media']['display_url'] # ok 
                if (len(data['graphql']['shortcode_media']['edge_media_to_caption']['edges'])>0):
                    text=data['graphql']['shortcode_media']['edge_media_to_caption']['edges'][0]['node']['text'] #ok
                else :
                    text=""
                hashtags=Hashtags_From_Text(text)
                posts_data.append({'Shortcode': shortcode,'Likes': likes,'Hashtags': hashtags,'Number of com\'s' : number_coms ,'Com\'s': coms ,'Text': text,'Url_img' : url_img,'Location': location})
                r.status_code=404
            else :
                failed_requests+=1
        time.sleep(10)
                
    #print("Requests successful : ",successful_requests)
    #print("Requests failed : ",failed_requests)
    print("Kicked requests : ",kicked_requests)
    with open(outfile_name+'_posts_data.json', 'w') as outfile:
        json.dump(posts_data, outfile) # save to json
    print(outfile_name+'_posts_data.json has been created')
    df=pd.read_json(outfile_name+"_posts_data.json")
    df.to_csv(outfile_name+'_data_results.csv')
    print(outfile_name+'_data_results.csv has been created')

In [6]:
def Posts_Data_From_Tag(tag,number_of_posts,outfile_name,minimum_likes,request_time_limit):
    end_cursor = '' # empty for the 1st page
    print("Start retrieving posts data from hashtag: ",tag,)
    print("Number of posts requested: ",number_of_posts)
    successful_requests=0
    failed_requests=0
    kicked_requests=0
    kicked_limit=10
    posts_data=[]
    currentDT = datetime.datetime.now()
    currentDT_month=str(currentDT.month)
    currentDT_day=str(currentDT.day)
    pages_retrieved = 0
    posts_retrieved=0
    if (len(currentDT_month)==1):
        currentDT_month="0"+currentDT_month
    if (len(currentDT_day)==1):
        currentDT_day="0"+currentDT_day
    
    currentDT=currentDT_day+currentDT_month+str(currentDT.year)
    request_time_limit+=2
    time_limit=time.time()
    while(posts_retrieved<number_of_posts):
        if ( time.time() - time_limit > request_time_limit) : 
            print("Exceeded time request to retrieve more posts")
            break
        url = "https://www.instagram.com/explore/tags/{0}/?__a=1&max_id={1}".format(tag, end_cursor)
        kicked_times=0
        while True and kicked_times<kicked_limit:
            try:
                r = requests.get(url,timeout = 2)
            except Exception as e:
                time.sleep(1)
                kicked_requests+=1
                kicked_times+=1
                continue
            break
        if ( r.status_code == 200):
            raw_data = r.text
            data = json.loads(raw_data.strip(),strict = False)
            #print(data)
            
            edge_hashtag= data['graphql']['hashtag']
            edge_hashtag_to_media=edge_hashtag['edge_hashtag_to_media']
            
            end_cursor = edge_hashtag_to_media ['page_info']['end_cursor'] # value for the next page
            
            edges = edge_hashtag_to_media ['edges'] # list with posts
            if(posts_retrieved==0):
                tag_id = edge_hashtag['id']
                tag_name = tag
                number_of_post_with_hashtag = edge_hashtag_to_media['count']
            page_time_start=time.time()
            for node in edges:
                if(posts_retrieved<number_of_posts):
                    post=node['node']
                    likes=post['edge_liked_by']['count']
                    if(likes>=minimum_likes):
                        #print("Retrieving post number",retrieved_post_number, "from page ",i)
                        owner_id = post['owner']['id']
                        timestamp = post['taken_at_timestamp']
                        shortcode=post['shortcode']
                        try :
                            number_coms=post['edge_media_to_comment']['count'] 
                        except :
                            number_coms=post['edge_media_parent_to_comment']['count']

                        coms=[]

                        url_img=post['display_url'] 
                        edges_media=post['edge_media_to_caption']['edges']
                        if (len(edges_media)>0):
                            text=edges_media[0]['node']['text'] 
                        else :
                            text=""

                        hashtags=Hashtags_From_Text(text)
                        mentions=Mentions_From_Text(text)
                        if( post["is_video"]==False):
                            img_description=post["accessibility_caption"]
                            if(img_description.find(":") !=-1):
                                img_description=img_description[img_description.index(":")+1:]
                            else:
                                img_description="Null"
                        else:
                            img_description="Video"
                        posts_data.append({'Tag_Id': tag_id,'Tag_Name': tag_name,'Extraction_Date':currentDT,'Posts_W_Hashtag':number_of_post_with_hashtag,'Owner_Id': owner_id,'Shortcode': shortcode,'Timestamp': timestamp,'Likes': likes,'Hashtags': hashtags,'Mentions': mentions,'Com\'s' : number_coms ,'Text': text,'Img_Description' : img_description,'Url_Img' : url_img})
                        r.status_code=404
                        posts_retrieved+=1
                        if (posts_retrieved%10==0):
                            print(posts_retrieved," Posts retrieved")
                        time_limit=time.time()
            pages_retrieved+=1
            page_time_end=time.time()
            #print("Page number :",pages_retrieved," retrieved - ","Time :",page_time_end-page_time_start)
        else :
            failed_requests+=1
                
    #print("Requests successful : ",successful_requests)
    #print("Requests failed : ",failed_requests)
    print("Kicked requests : ",kicked_requests)
    
    with open("#"+outfile_name+'_posts_data.json', 'w') as outfile:
        json.dump(posts_data, outfile) # save to json
    print("#"+outfile_name+'_posts_data.json has been created')
 
    #post = posts_data[0].keys()
   # with open("#"+outfile_name+'_posts_data.csv', 'w') as outfile:
    #    dict_writer = csv.DictWriter(outfile, post)
    #    dict_writer.writeheader()
    #    dict_writer.writerows(posts_data)


    print("Process finished")

In [ ]:
url = "https://www.instagram.com/bakary.gassama8"

r = requests.get(url,timeout = 5)
print(r.text)